# Find similar strings with FuzzyWuzzy

There are different ways to make data dirty, and inconsistent data entry is one of them. Inconsistent values are even worse than duplicates, and sometimes difficult to detect.<br>
In this notebook, I apply [FuzzyWuzzy](https://github.com/seatgeek/fuzzywuzzy) package to find similar ramen brand names in a ramen review dataset.<br>
Data source: [The ramen rater Big list](https://www.theramenrater.com/resources-2/the-list/)

## Import

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

## Overview of the dataset

In [2]:
# Load in the excel file
ramen = pd.read_excel('The-Ramen-Rater-The-Big-List-1-3400-Current-As-Of-Jan-25-2020.xlsx')

In [3]:
# Display the first columns
ramen.head()

,Review #,Brand,Variety,Style,Country,Stars
0,3400,EDO Pack,Kumamoto Flavour Noodles,Cup,Hong Kong,1
1,3399,Pan Mee,Goreng Dried Chili Shrimp Flavour,Pack,Malaysia,5
2,3398,Paldo,King Lid Ramen Noodle Soup,Pack,South Korea,5
3,3397,Nissin Miojo,Cremoso Carno Com Chili,Pack,Brazil,2
4,3396,Samyang Foods,Cham Ramen Big Bowl,Bowl,South Korea,2.25


In [4]:
# Check data type and null values
ramen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3400 entries, 0 to 3399
Data columns (total 6 columns):
Review #    3400 non-null int64
Brand       3400 non-null object
Variety     3400 non-null object
Style       3400 non-null object
Country     3400 non-null object
Stars       3400 non-null object
dtypes: int64(1), object(5)
memory usage: 159.5+ KB


In [5]:
# Remove leading and trailing spaces in each string value
ramen['Brand'] = ramen['Brand'].str.strip()
ramen['Variety'] = ramen['Variety'].str.strip()
ramen['Style'] = ramen['Style'].str.strip()
ramen['Country'] = ramen['Country'].str.strip()

Let's check number of unique values of Brand, Style, and Country

In [6]:
ramen['Brand'].nunique()

499

In [7]:
ramen['Style'].nunique()

8

In [8]:
ramen['Country'].nunique()

48

In [9]:
ramen['Country'].unique()

array(['Hong Kong', 'Malaysia', 'South Korea', 'Brazil', 'Japan',
       'United States', 'Taiwan', 'Mexico', 'China', 'Canada', 'Peru',
       'Spain', 'Thailand', 'Indonesia', 'Portugal', 'United Kingdom',
       'Italy', 'Russia', 'Singapore', 'Philippines', 'Bangladesh',
       'France', 'Australia', 'Vietnam', 'New Zealand', 'Ukraine',
       'Sarawak', 'Netherlands', 'Poland', 'Germany', 'USA', 'India',
       'Phlippines', 'Nigeria', 'Ghana', 'Hungary', 'Fiji', 'Pakistan',
       'Nepal', 'UK', 'Myanmar', 'Cambodia', 'Finland', 'Sweden',
       'Colombia', 'Estonia', 'Holland', 'Dubai'], dtype=object)

I would like to know if one brand can have multiple manufacturers in different countries.

In [10]:
brand_country = ramen['Brand'] +' '+ ramen['Country']
brand_country.nunique()

574

574 is greater than 499 brands, so yes, one brand can have different country values. Next, we need to get the list of unique brand names.

In [11]:
unique_brand = ramen['Brand'].unique().tolist()
sorted(unique_brand)[:80]

['1 To 3 Noodles',
 '7 Select',
 '7 Select/Nissin',
 '7-Eleven / Nissin',
 'A-One',
 'A-Sha',
 'A-Sha Dry Noodle',
 'A1',
 'ABC',
 'Acecook',
 'Adabi',
 'Ah Lai',
 'Aji-no-men',
 'Ajinatori',
 'Ajinomoto',
 'Alhami',
 'Amianda',
 'Amino',
 "Annie Chun's",
 'Aroi',
 'Asia Gold',
 'Asian Thai Foods',
 'Atomy',
 'Authentically Asian',
 'Azami',
 'Baijia',
 'Baixiang Noodles',
 'Baltix',
 'Bamee',
 'Banzai',
 'Batchelors',
 'Best Wok',
 'Big Bon',
 'Binh Tay',
 'Blount',
 'Bon Go Jang',
 'Bonasia',
 'Boss',
 'Bowlfull',
 'Buitoni',
 'Burung Layang Terbang',
 'CJ CheilJedang',
 'COCO 2.0',
 "Campbell's",
 'Canton',
 'Cap Atoom Bulan',
 'CarJEN',
 "Chaudhary's Wai Wai",
 "Chef Nic's Noodles",
 'Chencun',
 'Chering Chang',
 'Chewy',
 'Chikara',
 'Chil Kab Farm Products',
 'China Best',
 "Ching's Secret",
 'Chitato',
 'Chongqing Suanlabao Food Co., Ltd.',
 'Chorip Dong',
 'ChoripDong',
 'Choumama',
 'Chuan Wei Wang',
 'Chuanbei',
 'Cigala',
 'Cintan',
 'Conimex',
 'Crystal Noodle',
 "Culley's"

We can see some suspicious names right at the beginning. Let's have some tests first.

## FuzzyWuzzy

FuzzyWuzzy has four scorer options to find the Levenshtein distance between two strings. In this example, I would check on the token sort ratio and the token set ratio, for I believe they are more suitable for this dataset which might have mixed words order and duplicated words.<br>
I would pick four brand names and find their similar names in the Brand column. Since we're matching the Brand column with itself, the result would always include the selected name with a score of 100.

### Token Sort Ratio
The token sort ratio scorer tokenizes the strings and cleans them by returning these strings to lower cases, removing punctuations, and then sorting them alphabetically. After that, it finds the Levenshtein distance and returns the similarity percentage.

In [12]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_sort_ratio)

[('7 Select', 100),
 ('7 Select/Nissin', 70),
 ('Jinbo Selection', 61),
 ('Seven & I', 53),
 ('Lele', 50)]

This result means that '7 Select/Nissin' has 70% similarity when referring to '7 Select'. Not bad if I set the threshold at 70% to get the pair of 7 Select - 7 Select/Nissin.

In [13]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_sort_ratio)

[('A-Sha', 100), ('Shan', 67), ('Nasoya', 55), ('Alhami', 55), ('Ah Lai', 55)]

Below 70%, there's no match for A-sha.

In [14]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_sort_ratio)

[('Acecook', 100),
 ('Vina Acecook', 74),
 ('Yatekomo', 53),
 ('Sahmyook', 53),
 ('Panco', 50)]

Still good at 70% threshold.

In [15]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_sort_ratio)

[("Chef Nic's Noodles", 100),
 ("Mr. Lee's Noodles", 71),
 ('Fantastic Noodles', 69),
 ('1 To 3 Noodles', 62),
 ("Mom's Dry Noodle", 59)]

Now, we have a problem here. Token sort ratio scorer will gets the wrong pair of Chef Nic's Noodles - Mr. Lee's Noodles if I set 70% threshold.

In [16]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_sort_ratio)

[('Chorip Dong', 100),
 ('ChoripDong', 95),
 ('Hi-Myon', 56),
 ('Mr. Udon', 56),
 ('Maison de Coree', 54)]

This one looks good enough.

### Token Set Ratio
The token set ratio scorer also tokenizes the strings, and follows processing steps just like the token sort ratio. Then it collects common tokens between two strings and performs pairwise comparisons to find the similarity percentage.

In [17]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_set_ratio)

[('7 Select', 100),
 ('7 Select/Nissin', 100),
 ('The Ramen Rater Select', 86),
 ('Jinbo Selection', 61),
 ('Seven & I', 53)]

Since the token set ratio is more flexible, the score has increased from 70% to 100% for 7 Select - 7 Select/Nissin.

In [18]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_set_ratio)

[('A-Sha', 100),
 ('A-Sha Dry Noodle', 100),
 ('Shan', 67),
 ('Nasoya', 55),
 ('Alhami', 55)]

Now we see A-Sha has another name as A-Sha Dry Noodle. And we can see this only by using token set ratio.

In [19]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_set_ratio)

[('Acecook', 100),
 ('Vina Acecook', 100),
 ('Yatekomo', 53),
 ('Sahmyook', 53),
 ('Panco', 50)]

This one got 100% just like the 7 Select case.

In [20]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_set_ratio)

[("Chef Nic's Noodles", 100),
 ('S&S', 100),
 ('Mr. Noodles', 82),
 ("Mr. Lee's Noodles", 72),
 ('Tseng Noodles', 70)]

This one got much worse when token set ratio returns 100% for the pair of Chef Nic's Noodles - S&S.

In [21]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_set_ratio)

[('Chorip Dong', 100),
 ('ChoripDong', 95),
 ('Hi-Myon', 56),
 ('Mr. Udon', 56),
 ('Maison de Coree', 54)]

We have the same result for this one.

Although the token set ratio is more flexible and can detect more similar strings than the token sort ratio, it might also bring in more wrong matches.

## Apply FuzzyWuzzy in one column

To avoid obtain wrong matches with high scores, I choose the token sort ratio as the scorer for this dataset.

In [22]:
#Create tuples of brand names, matched brand names, and the score
get_score = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_sort_ratio)]

In [23]:
#Create dataframe from the tuples
similarity = pd.DataFrame(get_score, columns=['brand','match','score'])
#Create represented name to exclude the same pairs later
similarity['sorted_brand'] = np.minimum(similarity['brand'], similarity['match'])
similarity

,brand,match,score,sorted_brand
0,EDO Pack,EDO Pack,100,EDO Pack
1,EDO Pack,Gau Do,57,EDO Pack
2,EDO Pack,Pan Mee,53,EDO Pack
3,EDO Pack,Plats Du Chef,48,EDO Pack
4,EDO Pack,Vedan,46,EDO Pack
...,...,...,...,...
2490,Westbrae,Westbrae,100,Westbrae
2491,Westbrae,Trader Joe's,60,Trader Joe's
2492,Westbrae,Tabate,57,Tabate
2493,Westbrae,Easybab,53,Easybab


In [24]:
high_score = similarity.loc[(similarity['score'] >= 80) & #Select pairs which have scores at least 80% 
                            (similarity['brand'] != similarity['match']) & #not include the brand name itself
                            (similarity['sorted_brand'] != similarity['match'])] #not include the same pair
high_score.sort_values(['brand'])

,brand,match,score,sorted_brand
1481,Bamee,Mamee,80,Bamee
2441,Chorip Dong,ChoripDong,95,Chorip Dong
966,Chuanbei,Wei Chuan,82,Chuanbei
1171,Fashion Food,Fashion Foods,96,Fashion Food
291,FitMee,iMee,80,FitMee
2125,Goku Uma,Goku-Uma,100,Goku Uma
1531,Higashimaru,Ishimaru,84,Higashimaru
946,KOKA,Koka,100,KOKA
731,Kiki,Kuriki,80,Kiki
590,Lau Liu Tou,Lau Liu tou,100,Lau Liu Tou


In [25]:
#Group by brand names and scores
#pd.set_option('display.max_rows', None)
high_score.groupby(['brand','score']).agg({'match': ', '.join}).sort_values(['score'], ascending=False)

,,match
brand,score,
Nissin Miojo,100,Nissin-Miojo
Lau Liu Tou,100,Lau Liu tou
World O Noodle,100,World O' Noodle
Unif / Tung-I,100,Unif Tung-I
TableMark,100,Tablemark
MyKuali,100,Mykuali
Mamee Shinsegae,100,Mamee-Shinsegae
Mamee / Shinsegae,100,"Mamee Shinsegae, Mamee-Shinsegae"
MAMA,100,Mama


Taking time to look at this table, from the score of 91 and below, we would need to make some checks to see if these pairs refer to the same brand. Since this dataset is small, we can do this by listing out records from each pair.

In [26]:
ramen.loc[(ramen['Brand'] == 'Souper') | (ramen['Brand'] == 'Super')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars
2583,817,Souper,Bowl Ramen Shrimp,Bowl,Taiwan,3.75
2987,413,Souper,Alimentary Paste Chow Mein,Pack,Taiwan,4.25
523,2877,Super,Cup Instant Noodle Seafood Flavour,Cup,Malaysia,2.5
592,2808,Super,Super Cup Signature White Curry,Cup,Malaysia,0.25
1610,1790,Super,Kimchi,Cup,Malaysia,4.5
1612,1788,Super,Curry Flavour Noodles,Cup,Malaysia,3
1615,1785,Super,Instant Noodles Black Pepper Crab Mi Goreng,Cup,Malaysia,3.5
1668,1732,Super,Seafood Flavour Noodles,Cup,Malaysia,3.25
1905,1495,Super,Instant Noodles Mee Goreng,Cup,Malaysia,3.75


For this pair, we see that these two brands come from different manufacturers, and there's also no similarity in their ramen types or styles. I would say that these brands are not the same.

In [27]:
ramen.loc[(ramen['Brand'] == 'Sura') | (ramen['Brand'] == 'Suraj')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars
2756,644,Sura,Knife Cut Rice Noodle,Bowl,South Korea,3.75
285,3115,Suraj,Masala Noodles,Pack,Canada,2


Sura and Suraj are two different brands.

In [28]:
ramen.loc[(ramen['Brand'] == 'Ped Chef') | (ramen['Brand'] == 'Red Chef')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars
637,2763,Ped Chef,Sesame Clear Soup Noodles,Pack,Malaysia,3.5
164,3236,Red Chef,Spicy Sakura Prawn Soup Rice Vermicelli & Noodles,Pack,Malaysia,5
198,3202,Red Chef,Pandan White Curry Noodle (New Recipe),Pack,Malaysia,5
638,2762,Red Chef,Pandan White Curry Noodles,Pack,Malaysia,5
639,2761,Red Chef,Green Tom Yum Soup Noodles,Pack,Malaysia,5
640,2760,Red Chef,Spicy Sakura Prawn Soup Noodles,Pack,Malaysia,5


Here, we only have one record of the Ped Chef brand, and we also see a pattern in its variety name in comparison to Red Chef brand. I'm pretty sure it should be Red Chef brand.

We can continue to check other pairs by the same method. From the threshold of 84% and below, we can ignore some pairs which are obviously different or we can make a quick check as above. To get more matches beyond the token sort ratio scorer result, we can apply the token set ratio, but all the mistyped names with scores of 95% and above should be replaced first.